# DATA TRANSFORMATION - 14_farmacovigilancia
## 0. Previos

Se cargan las bases y las librerías a utilizar.

In [158]:
# Librerías
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import numpy as np

import generic_funcions as gf

ruta_archivos = 'C:/Users/monic/documentos/ds4a/project/datos/'

## 1. Load

In [159]:
dcc = gf.diccionario_llaves()

In [160]:
modulo = 'Farmacovigilancia RAM'

In [161]:
base = gf.carga_datos( ruta = ruta_archivos, diccionario = dcc, modulo = modulo )
base.head()

,fecha,DEPARTAMENTO,MUNICIPIO,EPS,ETNIA,FECHA_NACIMIENTO,EDAD,TIPO_IDENTIFICACION,id,SEXO,...,EXISTEN_OTROS_FACTORES_PUEDAN_EXPLICAR_EVENTO,EVENTO_DESAPARECIO_DISMINUIR_SUSP_MED,PACIENTE_HABIA_PRESENTADO_MISMA_REACION_MEDIC,CAUSALIDAD_SEGUN_ALGORITMO_DE_NARANJO,EVITABILIDAD,GRAVEDAD,DS_PLAN_INTERVENCION,year,month,year_month
0,2013-08-05,ANTIOQUIA,GIRARDOTA,EPS/SURA,MESTIZO,1977-09-06,42,CC,1706844,F,...,SI,NO SABE,NO SABE,POSIBLE,NaN,LEVE,SE LE INDICA A LA PACIENTE USAR SUS MEDICAMENT...,2013,8,201308
1,2013-03-05,ANTIOQUIA,ITAGUI,EPS/SURA,MESTIZO,1972-09-09,47,CC,1452148,F,...,SI,NO,SI,POSIBLE,NO,MODERADA,SE EXPLICA IMPORTANCIA A LA PACIENTE DE NUNCA ...,2013,3,201303
2,2013-03-06,ANTIOQUIA,MEDELLIN,EPS/SURA,MESTIZO,1966-10-02,53,CC,730790,F,...,SI,NO SABE,NO,POSIBLE,NO,MODERADA,SE COMUNICARA CASO AL MEDICO TRATANTE,2013,3,201303
3,2013-03-06,ANTIOQUIA,RIONEGRO,EPS/SURA,MESTIZO,1982-12-24,37,CC,738900,F,...,SI,NO SABE,SI,POSIBLE,NO,MODERADA,CONTINUAR EN SFT,2013,3,201303
4,2013-08-15,ANTIOQUIA,BELLO,EPS/SURA,MESTIZO,1974-08-21,45,CC,1479905,M,...,NaN,NO SABE,NO SABE,POSIBLE,NO,LEVE,PACIENTE QUE ENTIENDE RIESGO BENEFICIO DE LA A...,2013,8,201308


## 2. Transform

* Kind of reaction

In [162]:
base['indicacion_cat'] = base['INDICACION'].apply( lambda x : 'asma' if 'ASMA' in x else 'otra' )
base.groupby('indicacion_cat')['INDICACION'].count()

indicacion_cat
asma    225
otra     84
Name: INDICACION, dtype: int64

* Evolution

In [163]:
variable = 'EVOLUCION'
base.groupby(variable)['INDICACION'].count()

EVOLUCION
AUN SIN RECUPERACION       132
RECUPERADO CON SECUELAS      3
RECUPERADO SIN SECUELAS    173
Name: INDICACION, dtype: int64

In [164]:
def evolution_category( x ):
    category = ''
    if x == 'RECUPERADO SIN SECUELAS' :
        category = 'sin'
    elif x == 'RECUPERADO CON SECUELAS' :
        category = 'con'
    else :
        category = 'aun'
    return category
        
base['evolution_cat'] =  base['EVOLUCION'].apply( evolution_category )
base.head()

,fecha,DEPARTAMENTO,MUNICIPIO,EPS,ETNIA,FECHA_NACIMIENTO,EDAD,TIPO_IDENTIFICACION,id,SEXO,...,PACIENTE_HABIA_PRESENTADO_MISMA_REACION_MEDIC,CAUSALIDAD_SEGUN_ALGORITMO_DE_NARANJO,EVITABILIDAD,GRAVEDAD,DS_PLAN_INTERVENCION,year,month,year_month,indicacion_cat,evolution_cat
0,2013-08-05,ANTIOQUIA,GIRARDOTA,EPS/SURA,MESTIZO,1977-09-06,42,CC,1706844,F,...,NO SABE,POSIBLE,NaN,LEVE,SE LE INDICA A LA PACIENTE USAR SUS MEDICAMENT...,2013,8,201308,asma,sin
1,2013-03-05,ANTIOQUIA,ITAGUI,EPS/SURA,MESTIZO,1972-09-09,47,CC,1452148,F,...,SI,POSIBLE,NO,MODERADA,SE EXPLICA IMPORTANCIA A LA PACIENTE DE NUNCA ...,2013,3,201303,otra,sin
2,2013-03-06,ANTIOQUIA,MEDELLIN,EPS/SURA,MESTIZO,1966-10-02,53,CC,730790,F,...,NO,POSIBLE,NO,MODERADA,SE COMUNICARA CASO AL MEDICO TRATANTE,2013,3,201303,asma,sin
3,2013-03-06,ANTIOQUIA,RIONEGRO,EPS/SURA,MESTIZO,1982-12-24,37,CC,738900,F,...,SI,POSIBLE,NO,MODERADA,CONTINUAR EN SFT,2013,3,201303,asma,sin
4,2013-08-15,ANTIOQUIA,BELLO,EPS/SURA,MESTIZO,1974-08-21,45,CC,1479905,M,...,NO SABE,POSIBLE,NO,LEVE,PACIENTE QUE ENTIENDE RIESGO BENEFICIO DE LA A...,2013,8,201308,otra,sin


* EVENTO_DESAPARECIO_DISMINUIR_SUSP_MED

In [165]:
variable = 'EVENTO_DESAPARECIO_DISMINUIR_SUSP_MED'
base.groupby(variable)['INDICACION'].count()

EVENTO_DESAPARECIO_DISMINUIR_SUSP_MED
NO          85
NO SABE    195
SI          28
Name: INDICACION, dtype: int64

Dado que sólo hay tres categorías y todas informativas, se dejan tal cual (por ahora).

In [166]:
variable = 'desaparece_cat'
base[variable] = base['EVENTO_DESAPARECIO_DISMINUIR_SUSP_MED'].apply( lambda x : str(x).lower() )
base.groupby(variable)['INDICACION'].count()

desaparece_cat
nan          1
no          85
no sabe    195
si          28
Name: INDICACION, dtype: int64

* PACIENTE_HABIA_PRESENTADO_MISMA_REACION_MEDIC

In [167]:
variable = 'PACIENTE_HABIA_PRESENTADO_MISMA_REACION_MEDIC'
base.groupby(variable)['INDICACION'].count()

PACIENTE_HABIA_PRESENTADO_MISMA_REACION_MEDIC
NO          46
NO SABE     94
SI         168
Name: INDICACION, dtype: int64

In [168]:
variable = 'misma_cat'
base[variable] = base['PACIENTE_HABIA_PRESENTADO_MISMA_REACION_MEDIC'].apply( lambda x : str(x).lower() )
base.groupby(variable)['INDICACION'].count()

misma_cat
nan          1
no          46
no sabe     94
si         168
Name: INDICACION, dtype: int64

* CAUSALIDAD_SEGUN_ALGORITMO_DE_NARANJO

In [169]:
variable = 'CAUSALIDAD_SEGUN_ALGORITMO_DE_NARANJO'
base.groupby(variable)['INDICACION'].count()

CAUSALIDAD_SEGUN_ALGORITMO_DE_NARANJO
DEFINIDA     16
DUDOSA        1
POSIBLE     195
PROBABLE     96
Name: INDICACION, dtype: int64

In [170]:
def causalidad_category( x ):
    category = ''
    if x == 'DEFINIDA' :
        category = 'def'
    else :
        category = 'otra'
    return category
        
base['causalidad_cat'] =  base['CAUSALIDAD_SEGUN_ALGORITMO_DE_NARANJO'].apply( causalidad_category )
base.head()

,fecha,DEPARTAMENTO,MUNICIPIO,EPS,ETNIA,FECHA_NACIMIENTO,EDAD,TIPO_IDENTIFICACION,id,SEXO,...,GRAVEDAD,DS_PLAN_INTERVENCION,year,month,year_month,indicacion_cat,evolution_cat,desaparece_cat,misma_cat,causalidad_cat
0,2013-08-05,ANTIOQUIA,GIRARDOTA,EPS/SURA,MESTIZO,1977-09-06,42,CC,1706844,F,...,LEVE,SE LE INDICA A LA PACIENTE USAR SUS MEDICAMENT...,2013,8,201308,asma,sin,no sabe,no sabe,otra
1,2013-03-05,ANTIOQUIA,ITAGUI,EPS/SURA,MESTIZO,1972-09-09,47,CC,1452148,F,...,MODERADA,SE EXPLICA IMPORTANCIA A LA PACIENTE DE NUNCA ...,2013,3,201303,otra,sin,no,si,otra
2,2013-03-06,ANTIOQUIA,MEDELLIN,EPS/SURA,MESTIZO,1966-10-02,53,CC,730790,F,...,MODERADA,SE COMUNICARA CASO AL MEDICO TRATANTE,2013,3,201303,asma,sin,no sabe,no,otra
3,2013-03-06,ANTIOQUIA,RIONEGRO,EPS/SURA,MESTIZO,1982-12-24,37,CC,738900,F,...,MODERADA,CONTINUAR EN SFT,2013,3,201303,asma,sin,no sabe,si,otra
4,2013-08-15,ANTIOQUIA,BELLO,EPS/SURA,MESTIZO,1974-08-21,45,CC,1479905,M,...,LEVE,PACIENTE QUE ENTIENDE RIESGO BENEFICIO DE LA A...,2013,8,201308,otra,sin,no sabe,no sabe,otra


* GRAVEDAD

In [171]:
variable = 'GRAVEDAD'
base.groupby(variable)['id'].count()

GRAVEDAD
LEVE        248
MODERADA     58
SEVERA        2
Name: id, dtype: int64

In [172]:
def gravedad_category( x ):
    category = ''
    if x == 'LEVE' :
        category = 'leve'
    else :
        category = 'mod'
    return category
        
base['gravedad_cat'] =  base['GRAVEDAD'].apply( gravedad_category )
base.head()

,fecha,DEPARTAMENTO,MUNICIPIO,EPS,ETNIA,FECHA_NACIMIENTO,EDAD,TIPO_IDENTIFICACION,id,SEXO,...,DS_PLAN_INTERVENCION,year,month,year_month,indicacion_cat,evolution_cat,desaparece_cat,misma_cat,causalidad_cat,gravedad_cat
0,2013-08-05,ANTIOQUIA,GIRARDOTA,EPS/SURA,MESTIZO,1977-09-06,42,CC,1706844,F,...,SE LE INDICA A LA PACIENTE USAR SUS MEDICAMENT...,2013,8,201308,asma,sin,no sabe,no sabe,otra,leve
1,2013-03-05,ANTIOQUIA,ITAGUI,EPS/SURA,MESTIZO,1972-09-09,47,CC,1452148,F,...,SE EXPLICA IMPORTANCIA A LA PACIENTE DE NUNCA ...,2013,3,201303,otra,sin,no,si,otra,mod
2,2013-03-06,ANTIOQUIA,MEDELLIN,EPS/SURA,MESTIZO,1966-10-02,53,CC,730790,F,...,SE COMUNICARA CASO AL MEDICO TRATANTE,2013,3,201303,asma,sin,no sabe,no,otra,mod
3,2013-03-06,ANTIOQUIA,RIONEGRO,EPS/SURA,MESTIZO,1982-12-24,37,CC,738900,F,...,CONTINUAR EN SFT,2013,3,201303,asma,sin,no sabe,si,otra,mod
4,2013-08-15,ANTIOQUIA,BELLO,EPS/SURA,MESTIZO,1974-08-21,45,CC,1479905,M,...,PACIENTE QUE ENTIENDE RIESGO BENEFICIO DE LA A...,2013,8,201308,otra,sin,no sabe,no sabe,otra,leve


Variables de interés:

In [173]:
variables = ['indicacion_cat','evolution_cat','desaparece_cat','misma_cat','causalidad_cat','gravedad_cat']
ids = ['id','year','month']
base_ = base[ ids + variables]
base_['dum'] = 1
base_.head()

<ipython-input-173-1958933cb4c7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_['dum'] = 1


,id,year,month,indicacion_cat,evolution_cat,desaparece_cat,misma_cat,causalidad_cat,gravedad_cat,dum
0,1706844,2013,8,asma,sin,no sabe,no sabe,otra,leve,1
1,1452148,2013,3,otra,sin,no,si,otra,mod,1
2,730790,2013,3,asma,sin,no sabe,no,otra,mod,1
3,738900,2013,3,asma,sin,no sabe,si,otra,mod,1
4,1479905,2013,8,otra,sin,no sabe,no sabe,otra,leve,1


In [174]:
# pivot table to have separate variables, nans replaced with zero
def pivot_var( df, col ):
    df_pivot = df.pivot_table(index=ids, columns=[col], values='dum', aggfunc=np.sum).reset_index().fillna(0)
    return df_pivot

In [175]:
# Indicacion

base_ind = pivot_var( base_, 'indicacion_cat' )
base_ind['total'] = base_ind['asma'] + base_ind['otra']
base_ind = base_ind[ ids  + ['asma','total'] ]
pre = dcc[modulo]['prefi'] + '_' + 'rea_'
base_ind.columns = [ pre + s for s in base_ind.columns]
base_ind.rename( columns={ pre + 'id':'id', pre + 'year':'year', pre + 'month':'month' }, inplace=True)
base_ind.head()

,id,year,month,far_rea_asma,far_rea_total
0,525540,2019,12,1.0,1.0
1,530049,2016,9,1.0,1.0
2,535255,2016,11,1.0,1.0
3,535255,2019,10,0.0,1.0
4,539423,2016,9,1.0,1.0


In [176]:
# Evolucion

base_evo = pivot_var( base_, 'evolution_cat' )
base_evo = base_evo[ ids  +['aun','con'] ]
pre = dcc[modulo]['prefi'] + '_' + 'evo_'
base_evo.columns = [ pre + s for s in base_evo.columns]
base_evo.rename( columns={ pre + 'id':'id', pre + 'year':'year', pre + 'month':'month' }, inplace=True)
base_evo.head()

,id,year,month,far_evo_aun,far_evo_con
0,525540,2019,12,1.0,0.0
1,530049,2016,9,0.0,0.0
2,535255,2016,11,0.0,0.0
3,535255,2019,10,0.0,0.0
4,539423,2016,9,0.0,0.0


In [177]:
# Desaparece

base_des = pivot_var( base_, 'desaparece_cat' )
base_des = base_des[ ids  +['no','si'] ]
pre = dcc[modulo]['prefi'] + '_' + 'des_'
base_des.columns = [ pre + s for s in base_des.columns]
base_des.rename( columns={ pre + 'id':'id', pre + 'year':'year', pre + 'month':'month' }, inplace=True)

base_des.head()

,id,year,month,far_des_no,far_des_si
0,525540,2019,12,0.0,0.0
1,530049,2016,9,1.0,0.0
2,535255,2016,11,1.0,0.0
3,535255,2019,10,0.0,1.0
4,539423,2016,9,1.0,0.0


In [178]:
# Misma

base_mis = pivot_var( base_, 'misma_cat' )
base_mis = base_mis[ ids  +['si'] ]
pre = dcc[modulo]['prefi'] + '_' + 'mis_'
base_mis.columns = [ pre + s for s in base_mis.columns]
base_mis.rename( columns={ pre + 'id':'id', pre + 'year':'year', pre + 'month':'month' }, inplace=True)

base_mis.head()

,id,year,month,far_mis_si
0,525540,2019,12,0.0
1,530049,2016,9,0.0
2,535255,2016,11,1.0
3,535255,2019,10,0.0
4,539423,2016,9,1.0


In [179]:
# Causalidad

base_cau = pivot_var( base_, 'causalidad_cat' )
pre = dcc[modulo]['prefi'] + '_' + 'cau_'
base_cau.columns = [ pre + s for s in base_cau.columns]
base_cau.rename( columns={ pre + 'id':'id', pre + 'year':'year', pre + 'month':'month' }, inplace=True)

base_cau.head()

,id,year,month,far_cau_def,far_cau_otra
0,525540,2019,12,0.0,1.0
1,530049,2016,9,0.0,1.0
2,535255,2016,11,0.0,1.0
3,535255,2019,10,0.0,1.0
4,539423,2016,9,0.0,1.0


In [180]:
# Gravedad

base_gra = pivot_var( base_, 'gravedad_cat' )
pre = dcc[modulo]['prefi'] + '_' + 'gra_'
base_gra.columns = [ pre + s for s in base_gra.columns]
base_gra.rename( columns={ pre + 'id':'id', pre + 'year':'year', pre + 'month':'month' }, inplace=True)

base_gra.head()

,id,year,month,far_gra_leve,far_gra_mod
0,525540,2019,12,1.0,0.0
1,530049,2016,9,1.0,0.0
2,535255,2016,11,1.0,0.0
3,535255,2019,10,1.0,0.0
4,539423,2016,9,1.0,0.0


## 3. Merge with ids table

In [181]:
ids_mensual = gf.base_ids_mensual( ruta_archivos )

submodulos = {'ind': { 'base' : base_ind},
              'evo': { 'base' : base_evo},
              'des': { 'base' : base_des},
              'mis': { 'base' : base_mis},
              'cau': { 'base' : base_cau},
              'gra': { 'base' : base_gra}}

base_final_far = ids_mensual
for key in submodulos:
    base_final_far = base_final_far.merge( submodulos[key]['base'].head() , how='left')

base_final_far.tail()

,id,year,month,far_rea_asma,far_rea_total,far_evo_aun,far_evo_con,far_des_no,far_des_si,far_mis_si,far_cau_def,far_cau_otra,far_gra_leve,far_gra_mod
53275,1999629,2020,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53276,1999629,2020,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53277,1999629,2020,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53278,1999629,2020,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53279,1999629,2020,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [182]:
base_final_far.shape

(53280, 14)